# Proyecto 6: Sistemas de recomendación

## Dataset

El dataset que vamos a usar en este proyecto es el del proyecto [MovieLens](http://grouplens.org/datasets/movielens/). Estos son relevados por el grupo de investigación GroupLens de la Universidad de Minnesota. Vamos a usar la versión 100K.

Está basado en un sistema de rating de 5 estrellas y texto libre para tags a partir del servicio de recomendaciones MovieLens. Contiene 100.000 ratings para 1.682 películas. Los datos fueron creados por 943 usuarios.

En el [Readme](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt) del dataset se puede encontrar información adicional. A continuación vamos a describir cada uno de ellos.

### Ratings (u.data)

Cada línea representa el rating para una película para un usuario. Los ratings están basados en un sistema de 5 estrellas, con una salto de 0.5 estrellas. Los timestamps están expresados en segundos desde la medianoche de 01/1970 (UTC). El formato es:

`user id | item id | rating | timestamp`

### Usuarios (u.users)

Cada línea del archivo representa un usuario. Se incluye información demográfica. El formato es:

`user id | age | gender | occupation | zip code`

### Películas (u.item)

Cada línea corresponde a una película. Puede haber inconsistencias en los nombres. El formato es:

`movie id | movie title | release date | video release date | IMDb URL | generos`



Las columnas con los generos son binarias indicando la presencia o ausencia de cada uno en la película. Los géneros pueden ser:

* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western
* unknown

## Experimentación

Durante todo este proyecto vamos a utilizar la librería `surprise` para llevar a cabo las experimentaciones.

Como se mencionó en clase, las implementaciones en esta librería tiene algunas mejoras respecto de las estrategias convencionales como SVD. Esto se traduce en una mayor cantidad de parámetros para optimizar.

Vamos a levantar los datos. En primer lugar, definamos el formato para poder leer los datos.

**1) Utilizar** el `Reader` con el parámetro `line_format` igual a `'user item rating timestamp'` y `sep` como `'\t'`. Guardarlo en una variable `reader`.

In [2]:

from surprise import Dataset
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep='\t')


Utilizando este `reader` cargar los datos:

In [3]:
data = Dataset.load_from_file('dataset/ml-100k/u.data',reader=reader)

In [4]:
import pandas as pd
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('dataset/ml-100k/u.user', sep='|', names=u_cols,
                    encoding='latin-1', parse_dates=True) 

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('dataset/ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1')

m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('dataset/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

movie_ratings = pd.merge(movies, ratings)
df = pd.merge(movie_ratings, users)

df.head(5)

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076


In [5]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [6]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


__2) Ejecutar una corrida inicial usando cross validation y el algoritmo SVD (5 folds). Imprimir los RMSE de testing.__

In [8]:
from surprise import SVD
from surprise.model_selection import cross_validate

svd=SVD()
cv=cross_validate(svd,data,cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9366  0.9292  0.9384  0.9396  0.9411  0.9370  0.0041  
MAE (testset)     0.7401  0.7332  0.7370  0.7437  0.7426  0.7393  0.0038  
Fit time          4.62    4.70    5.10    4.79    4.52    4.74    0.20    
Test time         0.23    0.18    0.14    0.13    0.17    0.17    0.04    


In [9]:
print(cv['test_rmse'])

[0.93663831 0.92924228 0.938404   0.93957721 0.94111044]


__3) Evaluar el RMSE (promedio de cross validation) de testing de acuerdo a la cantidad de factores en SVD.__

In [10]:
import numpy as np
rmse_test_means = []

factors = [5,100,200,500,1000]
for factor in factors:
    algo = SVD(n_factors=factor)
    cv = cross_validate(algo, data, measures=['RMSE'],  cv=3, verbose=False)
    rmse_test_means.append(np.mean(cv['test_rmse']))
rmse_test_means

[0.943035022599591,
 0.948010817448473,
 0.9521673583302842,
 0.964141536453436,
 0.9802846996449176]

__4) Graficar los promedios RMSE del conjunto de testing de acuerdo a la cantidad de factores en SVD.__

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
plt.plot(factors, rmse_test_means, label='RMSE Testing')
plt.ylim((0.92, 1.0))
plt.legend(loc="best")
plt.title("RMSE para algoritmo SVD según cantidad de factores.")
plt.show()

__5) Usando GridSearch, encontrar el mejor estimador usando SVD con 500 factores. Imprimir el mejor rmse. Utilizar `refit` para obtener obtener el mejor estimador ya ajustado como resultado.__

Utilizar el espacio de parámetros:
* n_epochs: [5, 10]
* lr_all: [0.002, 0.005]
* reg_all: [0.4, 0.6]


In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors':[500],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3,refit=True)

gs.fit(data)

In [ ]:
print(gs.best_params['rmse'])

In [ ]:
print(gs.best_score['rmse'])

__6) Obtener una predicción para el usuario 321 para la película Titanic.__

In [ ]:
print(users.iloc[320])

In [ ]:
print(movies.iloc[312])

In [ ]:
movies.dtype

In [16]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.25)
algo.fit(trainset)
predictions = algo.test(testset)


In [17]:
predictions

[Prediction(uid='650', iid='195', r_ui=4.0, est=3.678435874948563, details={'was_impossible': False}),
 Prediction(uid='6', iid='494', r_ui=4.0, est=4.053437582935192, details={'was_impossible': False}),
 Prediction(uid='500', iid='781', r_ui=3.0, est=2.861798361639699, details={'was_impossible': False}),
 Prediction(uid='59', iid='447', r_ui=5.0, est=3.602318533401852, details={'was_impossible': False}),
 Prediction(uid='188', iid='1263', r_ui=3.0, est=3.7550667461903053, details={'was_impossible': False}),
 Prediction(uid='606', iid='79', r_ui=3.0, est=4.370535258029995, details={'was_impossible': False}),
 Prediction(uid='922', iid='1079', r_ui=1.0, est=2.47985841899803, details={'was_impossible': False}),
 Prediction(uid='868', iid='1480', r_ui=1.0, est=2.37540735328954, details={'was_impossible': False}),
 Prediction(uid='146', iid='301', r_ui=2.0, est=3.3954723208580164, details={'was_impossible': False}),
 Prediction(uid='345', iid='696', r_ui=3.0, est=3.0928797540259585, detail

In [24]:
algo.predict(str(321),str(313))

Prediction(uid='321', iid='313', r_ui=None, est=3.9417849643093787, details={'was_impossible': False})

In [19]:
#Se puede ver que el usuario no le puso rating a la pelicula Titanic pero el modelo precide que es de 3.53 estrellas

__7) Desarrollar una función que dado un usuario, obtenga la mejor predicción disponible (de acuerdo a nuestros datos de películas).__

In [45]:
col_names=['user','item','rating','timestamp']
ratings=pd.read_csv('dataset/ml-100k/u.data', sep='\t', names=col_names)

In [78]:
def mejor_predict(uid):
    watched=ratings[ratings['user']==uid]
    watched=list(watched['item'])
    not_watched = movies[~movies['movie_id'].isin(watched)].iloc[:,0:2]
    not_watched['predictions']=not_watched['movie_id'].apply(lambda x: gs.predict(str(uid),str(x)).est)
    not_watched.sort_values(by='predictions',ascending=False,inplace=True)
    not_watched.reset_index(drop=True,inplace=True)
    movie=not_watched['title'][0]
    rating=not_watched['predictions'][0]
    return print('La mejor predicción es {} con un rating de {}'.format(movie,rating))

In [ ]:
#Ejemplos 

In [79]:
mejor_predict(18)

La mejor predicción es Wallace & Gromit: The Best of Aardman Animation (1996) con un rating de 4.318016593343331


In [80]:
mejor_predict(313)

La mejor predicción es Close Shave, A (1995) con un rating de 4.243882073412289
